# Auteurs : BAUDET Quentin & LARMAILLARD-NOIREN Joris

Dans ce notebook, nous allons procéder à un pré-traitement approfondi des données issues du dataset de smartphones disponible sur Kaggle. L’objectif est de nettoyer, transformer et enrichir ces données afin de construire un jeu de données final, adapté à la conception d’un modèle d’intelligence artificielle dédié à la recommandation de smartphones.

Ce travail préparatoire est essentiel pour garantir la qualité des entrées du modèle, optimiser ses performances et assurer des recommandations pertinentes. Nous détaillerons chaque étape du traitement des données, depuis la gestion des valeurs manquantes jusqu’à la sélection des caractéristiques les plus informatives.

### Chargement des données

In [1]:
### Importation des modules
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import math

In [2]:
### Chargement des données
data = pd.read_csv('../data/processed/Smartphones_cleaned_dataset_completed.csv', sep=';')

data.head()

,brand_name,model,price,rating,has_5g,has_nfc,has_ir_blaster,processor_brand,num_cores,processor_speed,...,refresh_rate,num_rear_cameras,num_front_cameras,os,primary_camera_rear,primary_camera_front,extended_memory_available,extended_upto,resolution_width,resolution_height
0,apple,Apple iPhone 11,38999,73.0,False,True,False,bionic,6.0,2.65,...,60,2,1.0,ios,12.0,12.0,0,NaN,828,1792
1,apple,Apple iPhone 11 (128GB),46999,75.0,False,True,False,bionic,6.0,2.65,...,60,2,1.0,ios,12.0,12.0,0,NaN,828,1792
2,apple,Apple iPhone 11 Pro Max,109900,77.0,False,True,False,bionic,6.0,2.65,...,60,3,1.0,ios,12.0,12.0,0,NaN,1242,2688
3,apple,Apple iPhone 12,51999,74.0,True,True,False,bionic,6.0,3.10,...,60,2,1.0,ios,12.0,12.0,0,NaN,1170,2532
4,apple,Apple iPhone 12 (128GB),55999,75.0,True,True,False,bionic,6.0,3.10,...,60,2,1.0,ios,12.0,12.0,0,NaN,1170,2532


### Traitement des données

**Changement de la devise**

Comme nous l'avons vu précédemment dans le notebook `01_exploration`, nous allons changer la devise présente dans la colonne prix, pour la convertir en euro (€), car le dataset a été construit en Inde, et il faut donc adapter la devise utilisée en Europe.

In [3]:
### Conversion des prix roupie -> euro
data['price'] = data['price'] * 0.009966

### Arrondi des prix
data['price'] = data['price'].round(2)

data = data.rename(columns={'price': 'price (€)'})

data

,brand_name,model,price (€),rating,has_5g,has_nfc,has_ir_blaster,processor_brand,num_cores,processor_speed,...,refresh_rate,num_rear_cameras,num_front_cameras,os,primary_camera_rear,primary_camera_front,extended_memory_available,extended_upto,resolution_width,resolution_height
0,apple,Apple iPhone 11,388.66,73.0,False,True,False,bionic,6.0,2.65,...,60,2,1.0,ios,12.0,12.0,0,NaN,828,1792
1,apple,Apple iPhone 11 (128GB),468.39,75.0,False,True,False,bionic,6.0,2.65,...,60,2,1.0,ios,12.0,12.0,0,NaN,828,1792
2,apple,Apple iPhone 11 Pro Max,1095.26,77.0,False,True,False,bionic,6.0,2.65,...,60,3,1.0,ios,12.0,12.0,0,NaN,1242,2688
3,apple,Apple iPhone 12,518.22,74.0,True,True,False,bionic,6.0,3.10,...,60,2,1.0,ios,12.0,12.0,0,NaN,1170,2532
4,apple,Apple iPhone 12 (128GB),558.09,75.0,True,True,False,bionic,6.0,3.10,...,60,2,1.0,ios,12.0,12.0,0,NaN,1170,2532
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
974,xiaomi,Xiaomi Redmi Note 9 Pro,139.51,75.0,False,False,True,snapdragon,8.0,2.30,...,60,4,1.0,android,48.0,16.0,1,512.0,1080,2400
975,xiaomi,Xiaomi Redmi Note 9 Pro (4GB RAM + 128GB),143.90,77.0,False,False,True,snapdragon,8.0,2.30,...,60,4,1.0,android,48.0,16.0,1,512.0,1080,2400
976,xiaomi,Xiaomi Redmi Note 9 Pro Max,164.34,80.0,False,False,True,snapdragon,8.0,2.30,...,60,4,1.0,android,64.0,32.0,1,512.0,1080,2400
977,zte,ZTE Axon 30S,199.31,82.0,True,True,False,snapdragon,8.0,3.20,...,120,4,1.0,android,50.0,16.0,1,NaN,1080,2460


**Suppression de colonnes**

Dans cette section, nous allons supprimer certaines colonnes qui ne nous semblent pas pertinentes, ou déterminantes pour le choix d'un téléphone, notamment des fonctionnalités qui sont gadgets comme `has_ir_blaster`. Cela permettra d'être plus orienté vers la satisfaction de la clientèle.

In [4]:
### Suppression des colonnes
data = data.drop(columns=['has_ir_blaster', 'has_nfc'])

,brand_name,model,price (€),rating,has_5g,processor_brand,num_cores,processor_speed,battery_capacity,fast_charging_available,...,refresh_rate,num_rear_cameras,num_front_cameras,os,primary_camera_rear,primary_camera_front,extended_memory_available,extended_upto,resolution_width,resolution_height
0,apple,Apple iPhone 11,388.66,73.0,False,bionic,6.0,2.65,3110,0,...,60,2,1.0,ios,12.0,12.0,0,NaN,828,1792
1,apple,Apple iPhone 11 (128GB),468.39,75.0,False,bionic,6.0,2.65,3110,0,...,60,2,1.0,ios,12.0,12.0,0,NaN,828,1792
2,apple,Apple iPhone 11 Pro Max,1095.26,77.0,False,bionic,6.0,2.65,3500,1,...,60,3,1.0,ios,12.0,12.0,0,NaN,1242,2688
3,apple,Apple iPhone 12,518.22,74.0,True,bionic,6.0,3.10,2815,0,...,60,2,1.0,ios,12.0,12.0,0,NaN,1170,2532
4,apple,Apple iPhone 12 (128GB),558.09,75.0,True,bionic,6.0,3.10,2815,0,...,60,2,1.0,ios,12.0,12.0,0,NaN,1170,2532
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
974,xiaomi,Xiaomi Redmi Note 9 Pro,139.51,75.0,False,snapdragon,8.0,2.30,5020,1,...,60,4,1.0,android,48.0,16.0,1,512.0,1080,2400
975,xiaomi,Xiaomi Redmi Note 9 Pro (4GB RAM + 128GB),143.90,77.0,False,snapdragon,8.0,2.30,5020,1,...,60,4,1.0,android,48.0,16.0,1,512.0,1080,2400
976,xiaomi,Xiaomi Redmi Note 9 Pro Max,164.34,80.0,False,snapdragon,8.0,2.30,5020,1,...,60,4,1.0,android,64.0,32.0,1,512.0,1080,2400
977,zte,ZTE Axon 30S,199.31,82.0,True,snapdragon,8.0,3.20,4200,1,...,120,4,1.0,android,50.0,16.0,1,NaN,1080,2460


**Création d'une fonction de scoring pour catégoriser les téléphones**

Dans la section suivante, nous allons implémenter une fonction dans laquelle chaque ligne du dataset sera analysée, et pour chaque attribut des téléphones, nous allons attribuer un score qui sera utilisé après pour calculer un score final qui déterminera la catégorie du téléphone. Cela permettra d'affiner les futures recommandations du modèle, et de fournir des résultats qui pourront répondre aux besoins du client.

In [6]:
### Importation module
from typing import List, Tuple

### Fonction qui calcule un score total de chaque téléphone
def compute_total_score(phone) -> float:
    """
    Calcule un score de performance pour un téléphone donné 
    en fonction de ses caractéristiques techniques.

    Ce score permet ensuite de classifier le téléphone 
    en haut de gamme, milieu de gamme ou bas de gamme.
    :param phone: Correspond au téléphone (ligne) que l'on considère actuellement pour le calcul du score
    :return: Le score final du téléphone en fonction des différents critères d'évaluation
    """
    ### Fonction imbriquée qui généralise la logique de calcul des scores : pour les colonnes numériques
    def score_range(val, seuils: List[Tuple[float, float]]) -> float:
        for seuil, point in seuils:
            if val >= seuil and isinstance(val, (int, float)):
                return point
        return 0
    
    ### Fonction imbriquée qui généralise la logique de calcul des scores : pour les colonnes booléennes
    def score_boolean(val, point_if_true=0.5) -> float:
        return point_if_true if val else 0
    
    ### Variable qui stocke le score final
    score: float = 0
    
    ### Poids de chaque caractéristique des téléphones attribués selon l'importance de la caractéristique pour le choix du téléphone
    weights: List[float] = [0.25, 0.10, 0.20, 0.10, 0.15, 0.05, 0.05]
    
    ### Total des poids
    total_weight: float = sum(weights)
    
    ### Normalisation des poids
    normalized_weights: List[float] = [w / total_weight for w in weights]
    
    ### Calcul des scores : colonnes numériques
    score += score_range(phone['price (€)'], [(700, 1), (300, 0.5)]) * normalized_weights[0]
    score += score_range(phone['rating'], [(85, 1), (80, 0.5)]) * normalized_weights[1]
    score += score_range(phone['processor_speed'], [(2.8, 1), (2.4, 0.5)]) * normalized_weights[2]
    score += score_range(phone['battery_capacity'], [(4500, 1), (3500, 0.5)]) * normalized_weights[3]
    score += score_range(phone['internal_memory'], [(256, 1), (128, 0.5)]) * normalized_weights[4]
    score += score_range(phone['primary_camera_rear'], [(64, 1), (48, 0.5)]) * normalized_weights[5]
    
    ### Calcul des scores : colonnes booléennes
    score += score_boolean(phone['has_5g'], 1) * normalized_weights[6]
    
    return score
        

Nous pouvons voir également que la colonne `rating` possède des valeurs manquantes qui sont cruciales pour le calcul du score de chaque téléphone. Pour remplacer ces valeurs, nous allons calculer les valeurs médianes des `rating` par marque de téléphones, et les attribuer aux téléphones qui n'ont pas de note répertoriée.

In [7]:
### Calcul des médianes des `rating` par marque
median_per_brand = data.groupby('brand_name')['rating'].median()

data['rating'] = data.apply(
    lambda phone: median_per_brand[phone['brand_name']] if pd.isna(phone['rating']) else phone['rating'],
    axis=1
)

In [8]:
### Fonction de catégorisation des téléphones
def determine_range(telephone) -> str:
    """
    Calcule un score de performance pour un téléphone donné 
    en fonction de ses caractéristiques techniques.

    Ce score permet ensuite de classifier le téléphone 
    en haut de gamme, milieu de gamme ou bas de gamme.
     
    :param telephone: Prend en argument chaque téléphone (ligne) du dataset
    :return: Une chaîne de caractères qui indique la catégorie du téléphone
    """
    score: float = compute_total_score(telephone)
    
    ### Attribution des catégories
    if score >= 0.7:
        return "Haut de gamme"
    elif score >= 0.4:
        return "Milieu de gamme"
    else:
        return "Entrée de gamme"

In [9]:
### Application au dataset
data['range'] = data.apply(determine_range, axis=1)

data

,brand_name,model,price (€),rating,has_5g,has_nfc,has_ir_blaster,processor_brand,num_cores,processor_speed,...,num_rear_cameras,num_front_cameras,os,primary_camera_rear,primary_camera_front,extended_memory_available,extended_upto,resolution_width,resolution_height,gamme
0,apple,Apple iPhone 11,388.66,73.0,False,True,False,bionic,6.0,2.65,...,2,1.0,ios,12.0,12.0,0,NaN,828,1792,Entrée de gamme
1,apple,Apple iPhone 11 (128GB),468.39,75.0,False,True,False,bionic,6.0,2.65,...,2,1.0,ios,12.0,12.0,0,NaN,828,1792,Entrée de gamme
2,apple,Apple iPhone 11 Pro Max,1095.26,77.0,False,True,False,bionic,6.0,2.65,...,3,1.0,ios,12.0,12.0,0,NaN,1242,2688,Milieu de gamme
3,apple,Apple iPhone 12,518.22,74.0,True,True,False,bionic,6.0,3.10,...,2,1.0,ios,12.0,12.0,0,NaN,1170,2532,Milieu de gamme
4,apple,Apple iPhone 12 (128GB),558.09,75.0,True,True,False,bionic,6.0,3.10,...,2,1.0,ios,12.0,12.0,0,NaN,1170,2532,Milieu de gamme
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
974,xiaomi,Xiaomi Redmi Note 9 Pro,139.51,75.0,False,False,True,snapdragon,8.0,2.30,...,4,1.0,android,48.0,16.0,1,512.0,1080,2400,Entrée de gamme
975,xiaomi,Xiaomi Redmi Note 9 Pro (4GB RAM + 128GB),143.90,77.0,False,False,True,snapdragon,8.0,2.30,...,4,1.0,android,48.0,16.0,1,512.0,1080,2400,Entrée de gamme
976,xiaomi,Xiaomi Redmi Note 9 Pro Max,164.34,80.0,False,False,True,snapdragon,8.0,2.30,...,4,1.0,android,64.0,32.0,1,512.0,1080,2400,Entrée de gamme
977,zte,ZTE Axon 30S,199.31,82.0,True,True,False,snapdragon,8.0,3.20,...,4,1.0,android,50.0,16.0,1,NaN,1080,2460,Milieu de gamme


**Estimation du rapport qualité/prix**

Pour prendre en compte l'avis des utilisateurs sur les téléphones par rapport au prix de vente qu'ils ont, nous allons créer une nouvelle métrique `quality-price ratio`. Nous avons vu que pour une même note, par exemple 85, les prix peuvent varier de moins de `100 €` à plus de `1000 €`, ce qui montre que la note ne reflète pas uniquement le prix, mais elle peut aussi intégrer la qualité perçue, la marque, la fiabilité, ou encore l’expérience utilisateur.

Cependant, pour définir cette métrique, nous allons l'appliquer par segment de gamme. C'est-à-dire qu'un téléphone de haute gamme peut avoir, avec cette nouvelle métrique, un ratio qui peut être bas. Et, nous devons noter que, les téléphones haut de gamme ne sont pas destinés à être économique, il n'est pas pertinent d'estimer ce rapport pour cette gamme de téléphone. Donc, pour ce faire, nous allons créer des ratios par gamme. 